In [25]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import scipy as sp


- look at configs and see if trends visible
- compare with square
- correlated expectation values (read up)

Step 1: Define the Lattice Structure
- Lattice Shape (number indicates a dipole exists, 0 indicates no dipole)
- Spin states (each dipole will be represented by an entry in the lattice with a value of either +1 or -1)

Step 2: Hamiltonian Construction

- The hamiltonian describes the total energy of the system, accounting for interactions between neightboring spins. The enerrgy interaction between two neightboring spins is typically proportional to the product of their spin states (i.e. $H = -J\sum S_iS_j$, where $J$ is a coupling constant and $S_i$ and $S_j$ are the spin states of neighbors.)
- Once lattice is constructed, we need to filter out which dipoles are neighbors. For a 2D array, a dipole can interact with its left, right, top, below, top right, top left, bottom right, bottom left neighbors.

Step 3: Calculate total energy

- Use $H = -J\sum S_iS_j$ formula and sum up the energy contributions from every dipole in the lattice to determine the "total energy"

In [26]:
def lattice_generator(x, N, M):
    bits = N * M
    # Convert integer x to binary representation, zero-padded to fit N*M bits
    if len(bin(x)[2:]) > bits:
        sys.exit("Error: Maximum is greater than lattice shape. Please try a lower maximum.")
    return np.array([int(i) for i in bin(x)[2:].zfill(bits)])

def hamiltonian_boundary(spin_lattice, J):
    total_energy = 0
    rows, cols = spin_lattice.shape

    for i in range(rows):
        for j in range(cols):
            # Consider only if the current index is a spin
            if spin_lattice[i][j] != 0:
                # Neighbors with periodic boundary conditions
                neighbors = [
                    ((i + 0) % rows, (j + 1) % cols),  # Right
                    ((i + 1) % rows, (j + 0) % cols),  # Down
                    ((i + 1) % rows, (j + 1) % cols),  # Down-Right
                ]
                
                for ni, nj in neighbors:
                    total_energy += J * spin_lattice[i][j] * spin_lattice[ni][nj]

    return total_energy

# Lattice size
N = 3
M = 3
maximum = 511 #2^9-1

bit_string = lattice_generator(maximum, N, M) #101011110
spin_lattice = bit_string.reshape(N, M)
spin_lattice = np.where(spin_lattice == 0, -1, 1) # Convert 0 to 1 (up spin) and 1 to -1 (down spin)

J = 1
total_energy = hamiltonian_boundary(spin_lattice, J)

print("Spin Lattice:\n", spin_lattice)
print("\nTotal Energy (periodic boundary) :", total_energy)
print("\nBit String: ", bit_string)

Spin Lattice:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]

Total Energy (periodic boundary) : 27

Bit String:  [1 1 1 1 1 1 1 1 1]


In [27]:
# Lattice size
N = 6
M = 9
maximum = 5115413828

bit_string = lattice_generator(maximum, N, M)
spin_lattice = bit_string.reshape(N, M)
spin_lattice = np.where(spin_lattice == 0, -1, 1) # Convert 0 to 1 (up spin) and 1 to -1 (down spin)

J = 1
total_energy = hamiltonian_boundary(spin_lattice, J)

print("Spin Lattice:\n", spin_lattice)
print("\nTotal Energy (periodic boundary) :", total_energy)
print("\nBit String: ", bit_string)

Spin Lattice:
 [[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1  1  1 -1]
 [-1 -1 -1  1  1  1 -1 -1  1]
 [ 1  1 -1 -1 -1 -1 -1  1 -1]
 [ 1 -1  1 -1 -1 -1  1 -1 -1]]

Total Energy (periodic boundary) : 46

Bit String:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1
 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0]


In [ ]:
# Lattice size
N = 2
M = 2

# calculates 2^n - 1
maximum = 2**(M*N)-1
print(maximum)
bit_string = lattice_generator(maximum, N, M)
print("Max Bit String: ", bit_string)

def lowest_energy(N, M, maximum):
    bit_strings = []
    lattice_list = []
    count = 0
    minimum_energy = float('inf')  # Initialize to a large value
    
    for i in range(0,maximum+1):
        bit_string = lattice_generator(i, N, M)
        spin_lattice = bit_string.reshape(N, M)
        spin_lattice = np.where(spin_lattice == 0, -1, 1)  # Convert 0 to -1 (spin down), 1 to 1 (spin up)
        J = 1
        total_energy = hamiltonian_boundary(spin_lattice, J)

        # Check if the current configuration has lower or equal energy
        if total_energy < minimum_energy:
            minimum_energy = total_energy
            bit_strings = [bit_string]  # Reset with the new lowest
            lattice_list = [i]
            count = 1
        elif total_energy == minimum_energy:
            bit_strings.append(bit_string)
            lattice_list.append(i)
            count += 1
    
    return count, bit_strings, minimum_energy, lattice_list


count, bitstrings, minimum_energy, latticelist = lowest_energy(N,M,maximum)
#print(bitstrings)

print("The minimum energy found was %i. It was found %i times."%(minimum_energy, count))
print("The given bit string integer(s) were:", latticelist)
# print(latticelist)
# print(minimum_energy, count)

15
Max Bit String:  [1 1 1 1]
The minimum energy found was -4. It was found 6 times.
The given bit string integer(s) were: [3, 5, 6, 9, 10, 12]


In [29]:
def average_lattice(bitstrings, N, M):
    # Convert bitstrings into NumPy arrays and reshape each to (N, M)
    lattices = [bitstring.reshape(N, M) for bitstring in bitstrings]
    
    # Stack the lattices into a 3D array (each lattice as a new "layer")
    lattices_stack = np.array(lattices)
    
    # Compute the average across the first axis (over all configurations)
    average_lattice = np.mean(lattices_stack, axis=0)
    
    return average_lattice

# Example usage of the functions
count, bitstrings, minimum_energy, latticelist = lowest_energy(N, M, maximum)

print(f"The minimum energy found was {minimum_energy}. It was found {count} times.")
print("The given bit string integer(s) were:", latticelist)

# Compute the average lattice
average = average_lattice(bitstrings, N, M)

print("\nAverage Spin Lattice (averaged over all configurations with minimum energy):")
print(average)

The minimum energy found was -4. It was found 6 times.
The given bit string integer(s) were: [3, 5, 6, 9, 10, 12]

Average Spin Lattice (averaged over all configurations with minimum energy):
[[0.5 0.5]
 [0.5 0.5]]
